# Decoding of [7,4,2] linear block code using belief propagation algorithm




* Generating a [7,4,2] code

In [81]:
import numpy as np
ulist=[]
v=[]
v_i= 0
vlist=[]
for u1 in range(2):
    for u2 in range(2):
        for u3 in range(2):
            for u4 in range(2):
                u=[u1,u2,u3,u4]
                ulist.append(u)
                # print(u)
ulist = np.array(ulist)                
# print("U=",ulist)
G=[[0,0,0,0,1,0,1],[1,1,0,0,0,0,0],[0,0,1,0,0,1,0],[1,0,0,1,0,1,1]]
G = np.array(G)
# print(G)
for i in range(16):
    for n in range(len(G[0])):
        for k in range(len(G)):
            v_i = (v_i + ulist[i,k]*G[k,n])%2
        v.append(v_i)
        v_i=0
    vlist.append(v)
    v=[]
vlist=np.array(vlist)
for i in range(16):
    print("Symbol=",ulist[i,:],"\t Code=",vlist[i,:],"\n")

Symbol= [0 0 0 0] 	 Code= [0 0 0 0 0 0 0] 

Symbol= [0 0 0 1] 	 Code= [1 0 0 1 0 1 1] 

Symbol= [0 0 1 0] 	 Code= [0 0 1 0 0 1 0] 

Symbol= [0 0 1 1] 	 Code= [1 0 1 1 0 0 1] 

Symbol= [0 1 0 0] 	 Code= [1 1 0 0 0 0 0] 

Symbol= [0 1 0 1] 	 Code= [0 1 0 1 0 1 1] 

Symbol= [0 1 1 0] 	 Code= [1 1 1 0 0 1 0] 

Symbol= [0 1 1 1] 	 Code= [0 1 1 1 0 0 1] 

Symbol= [1 0 0 0] 	 Code= [0 0 0 0 1 0 1] 

Symbol= [1 0 0 1] 	 Code= [1 0 0 1 1 1 0] 

Symbol= [1 0 1 0] 	 Code= [0 0 1 0 1 1 1] 

Symbol= [1 0 1 1] 	 Code= [1 0 1 1 1 0 0] 

Symbol= [1 1 0 0] 	 Code= [1 1 0 0 1 0 1] 

Symbol= [1 1 0 1] 	 Code= [0 1 0 1 1 1 0] 

Symbol= [1 1 1 0] 	 Code= [1 1 1 0 1 1 1] 

Symbol= [1 1 1 1] 	 Code= [0 1 1 1 1 0 0] 



* Generating messages

    A binary symmetric channel with error probability 't' causing at most **one** error in received codeword.
    <img src="Images/BSC.png">

In [82]:
C = "0111100"    #Original codeword
R = "1111100"    #Received codeword
t=theta=0.1      #Error probability

# Construct message for each node 
def log(x):
    log= -np.log2(x)
    return log

x = np.empty([7,2])
for i in range(7):
    if R[i]=="0":
        x[i] = np.array([log(1-t),log(t)])
    else:
        x[i] = np.array([log(t),log(1-t)])


In [83]:
# ####################### 0,1 -> 013 #################
# u0_013 = x[0]; u1_013 = x[1]
# ####################### 2,5 -> 235 #################
# u5_235 = x[5]; u2_235 = x[2]
# ####################### 4,6 -> 346 #################
# u6_346 = x[6]; u4_346 = x[4]

u013_3 = u235_3 = u346_3 = np.zeros(2)
u013_0 = u013_1 = u235_2 = u235_5 = u346_4 = u346_6 = np.zeros(2)
x3_final = np.zeros(2)

* Pass messages from leaf nodes to root node X3

In [84]:
def first_pass(a,b):
    tar = np.empty(2)
    for i3 in range(2):
        store_sum = []
        # print("for x3=",i3)
        for i0 in range(2):
            for i1 in range(2):
                if (i0+i1+i3)%2==0:
                    sum= x[a][i0]+x[b][i1]
                    store_sum.append(sum)
        
        # print("Min",store_sum," = ",min(store_sum))
        tar[i3] = min(store_sum)
    tar = tar - max(tar)    #Normalization
    return tar

* Pass messages backwards from root node to leaf nodes

In [85]:
def final_pass(n):
    tar= np.empty(2)
    global x3_final
    x3_final = x[3] + u013_3 + u235_3 + u346_3
    x3_final = x3_final - max(x3_final)  #Normalization

    for i0 in range(2):
        store_sum = []
        # print("for x0=",i0)
        for i3 in range(2):
            for i1 in range(2):
                if (i0+i1+i3)%2==0:
                    if n == 0:
                        sum= x[3][i3] + u235_3[i3] + u346_3[i3] +x[1][i1]
                    elif n == 1:
                        sum= x[3][i3] + u235_3[i3] + u346_3[i3] +x[0][i1]
                    elif n == 2:
                        sum= x[3][i3] + u013_3[i3] + u346_3[i3] +x[5][i1]
                    elif n == 5:
                        sum= x[3][i3] + u013_3[i3] + u346_3[i3] +x[2][i1]
                    elif n == 4:
                        sum= x[3][i3] + u013_3[i3] + u235_3[i3] +x[6][i1]
                    elif n == 6:
                        sum= x[3][i3] + u013_3[i3] + u235_3[i3] +x[4][i1]
                    store_sum.append(sum)
        # print("Min",store_sum)
        # print(min(store_sum))
        tar[i0]=min(store_sum)
    tar = tar - max(tar)  #Normalization
    return tar

* Fianl decoding of local domains:

In [86]:
def decoding():
    decoded_R = np.empty(7)
    x_final = np.empty([7,2])
    for i in range(7):
        if i==0:
            x_final[i] = x[i] + u013_0
            decoded_R[i] = np.argmin(x_final[i])
        elif i == 1:
            x_final[i] = x[i] + u013_1
            decoded_R[i] = np.argmin(x_final[i])
        elif i == 2:
            x_final[i] = x[i] + u235_2
            decoded_R[i] = np.argmin(x_final[i])
        elif i == 3:
            x_final[i] = x3_final
            decoded_R[i] = np.argmin(x_final[i])
        elif i == 5:
            x_final[i] = x[i] + u235_5
            decoded_R[i] = np.argmin(x_final[i])
        elif i == 4:
            x_final[i] = x[i] + u346_4
            decoded_R[i] = np.argmin(x_final[i])
        elif i == 6:
            x_final[i] = x[i] + u346_6
            decoded_R[i] = np.argmin(x_final[i])
    
    return(x_final,decoded_R)

Running the algorithm:

In [87]:
for i in range(100):
    # print("Iteration {}".format(i+1))

    u013_3 =  first_pass(0,1) 
    # print("013 -> 3 = ",u013_3,"\n")
    u235_3 =  first_pass(2,5)
    # print("235 -> 3 =",u235_3,"\n")
    u346_3 =  first_pass(4,6)
    # print("346 -> 3 =",u346_3,"\n")
    
    
    u013_0 =  final_pass(0)
    # print("013 -> 0 =",u013_0,"\n")
    u013_1 =  final_pass(1)
    # print("013 -> 1 =",u013_1,"\n")
    # print("3 ->013 = ", x3_final - u013_3)
    u235_2 =  final_pass(2)
    # print("235 -> 2 =",u235_2,"\n")
    u235_5 =  final_pass(5)
    # print("235 -> 5 =",u235_5,"\n")

    u346_4 =  final_pass(4)
    # print("346 -> 4 =",u346_4,"\n")
    u346_6 =  final_pass(6)
    # print("346 -> 6 =",u346_6,"\n")
    x_final, decoded_R = decoding()
    x = x_final
    


print("\n Final Costs =")
for i in range(7):
    print("x{} = ".format(i),x_final[i]) 
print("\n Original codeword = ",C)
print(" Received codeword = ",R)
print("\n Decoded codeword:", decoded_R)


 Final Costs =
x0 =  [0.15200309 0.15200309]
x1 =  [0.15200309 0.15200309]
x2 =  [ 3.32192809e+00 -4.01835733e+30]
x3 =  [ 0.00000000e+00 -8.03671466e+30]
x4 =  [ 3.32192809e+00 -4.01835733e+30]
x5 =  [-4.01835733e+30  3.32192809e+00]
x6 =  [-4.01835733e+30  3.32192809e+00]

 Original codeword =  0111100
 Received codeword =  1111100

 Decoded codeword: [0. 0. 1. 1. 1. 0. 0.]
